### O METAR, sigla de Meteorological Aerodrome Report, é o informe meteorológico regular de aeródromo e contém as seguintes informações na sequência:

- Grupos de identificação;  
- Vento à superfície;  
- Visibilidade horizontal; 
- Alcance visual na pista (quando houver); 
- Tempo presente; 
- Nuvens (ou visibilidade vertical, se for o caso); 
- Temperaturas do ar e do ponto de orvalho; 
- Pressão atmosférica (QNH); e  
- Informações suplementares de inclusão condicional sobre tempo recente, cortante do vento, temperatura da superfície do mar, estado do mar e, por Acordo Regional de Navegação Aérea, o estado da pista.

https://ajuda.decea.mil.br/base-de-conhecimento/como-decodificar-o-metar-e-o-speci/


pip install python-metar


In [1]:
from metar import Metar
from datetime import datetime, timedelta
import pandas as pd
import re

df_metar = pd.read_csv("./processed_data/metaf_bronze.csv")


In [2]:


def interpretaMetaR(metarBruto):

    metarBruto = metarBruto.replace('METAF', 'METAR')
    metarBruto = metarBruto.replace('\n', '')


    station = None
    type = None
    hora_formated = None
    temperature = None
    dew_point = None
    windDirection = None
    windVelocity = None
    visibility = None
    pressure = None
    weather = None
    sky = None
    skyFeet= None

    try: 

        obs = Metar.Metar(metarBruto[:-1])

        for item in obs.string().split('\n'):
            if("station" in item):
                station = item.split(" ")[1]
            if("type" in item):
                type = " ".join(item.split(" ")[1:])
            if("time" in item):
                timeZ = " ".join(item.split(" ")[1:])
                data_obj = datetime.strptime(timeZ, "%a %b %d %H:%M:%S %Y")
                nova_data_obj = data_obj - timedelta(hours=3)
                hora_formated = nova_data_obj.strftime("%a %b %d %H:%M:%S %Y")
            if("temperature" in item):
                temperature = item.split(" ")[1]
            if("dew point" in item):
                dew_point = item.split(" ")[2]
            if("wind" in item):
                wind = (item.split(" ")[1:])
                windDirection = wind[0]
                windVelocity = wind[2]
            if("visibility" in item):
                visibility = item.split(" ")[1]
            if("pressure" in item):
                pressure = item.split(" ")[1]
            if("weather" in item):
                weather = " ".join(item.split(" ")[1:])
            if("sky" in item):
                skyString = " ".join(item.split(" ")[1:])
                if(' at ' in skyString):
                    sky = ''.join(skyString.split(' at ')[0])
                else:
                    sky = ''.join(skyString.split(',')[0])
                
                allFeets = re.findall(r"(\d+)",skyString)
                if len(allFeets) > 0:
                    skyFeet = allFeets[0]
    except:
        pass

    return station, type, hora_formated, temperature,dew_point,windDirection,windVelocity,visibility,pressure,weather,sky,skyFeet
        



In [3]:
resultado_df =  df_metar['metaf'].apply(interpretaMetaR)
resultado_df =  pd.DataFrame(resultado_df.tolist(),columns=['station', 'type', 'metar_hora_formated', 'station','dew_point','windDirection','windVelocity','visibility','pressure','weather','sky','skyFeet'] )


metaR_bronze = pd.concat([df_metar, resultado_df], axis=1)


In [4]:
direction_to_degrees = {
    'N': 0,
    'NNE': 22.5,
    'NE': 45,
    'ENE': 67.5,
    'E': 90,
    'ESE': 112.5,
    'SE': 135,
    'SSE': 157.5,
    'S': 180,
    'SSW': 202.5,
    'SW': 225,
    'WSW': 247.5,
    'W': 270,
    'WNW': 292.5,
    'NW': 315,
    'NNW': 337.5,
    'calm': None,
    'variable': None,
}

metaR_bronze['windDirection'] = metaR_bronze['windDirection'].map(direction_to_degrees)

In [5]:
metaR_bronze = metaR_bronze.sort_values(by=['aero','metar_hora_formated'])
metaR_bronze = metaR_bronze.drop_duplicates()
metaR_bronze = metaR_bronze.dropna()


In [18]:
metaR_bronze.head(3)

,hora,metaf,aero,hora_formated,station,type,metar_hora_formated,station,dew_point,windDirection,windVelocity,visibility,pressure,weather,sky,skyFeet
5966,1661061600000,METAF SBBR 210600Z 09007KT 5000 BR FEW033 ...,SBBR,"Sunday, August 21, 2022 03:00:00",SBBR,"routine report, cycle 6 (automatic report)",Mon Aug 21 03:00:00 2023,12.0,9.0,90.0,7,5000,1022.0,mist,a few clouds,3300
5967,1661065200000,METAF SBBR 210700Z 07006KT 3000 BR SCT021 ...,SBBR,"Sunday, August 21, 2022 04:00:00",SBBR,"routine report, cycle 7 (automatic report)",Mon Aug 21 04:00:00 2023,11.0,9.0,67.5,6,3000,1020.0,mist,scattered clouds,2100
5968,1661068800000,METAF SBBR 210800Z 10006KT 2000 BR BKN025...,SBBR,"Sunday, August 21, 2022 05:00:00",SBBR,"routine report, cycle 8 (automatic report)",Mon Aug 21 05:00:00 2023,10.0,9.0,90.0,6,2000,1020.0,mist,broken clouds,2500


In [6]:
metaR_silverDummies = pd.get_dummies(metaR_bronze['sky'])
metaR_silver = pd.concat([metaR_bronze, metaR_silverDummies], axis=1)


In [7]:
def defineIndex(df):
    df = df.sort_values(by='hora_formated')
    df.set_index('hora_formated', inplace=True)
    df.index = pd.to_datetime(df.index, format='%A, %B %d, %Y %H:%M:%S')
    return df
    
def calculate_average(series):
    result = []
    for i in range(len(series) - 1):
        current_value = series.iloc[i]
        next_value = series.iloc[i + 1]
        average = (current_value + next_value) / 2
        result.append(average)
    return pd.Series(result, index=series.index[:-1])

def pipemediaTempoMetar(df, colunas):
    listSeries = []

    for coluna in colunas:
        df[coluna] = pd.to_numeric(df[coluna], errors='coerce')
        df_resampled_temp = df[coluna].resample('30T').mean()
        df_resampled_temp_filled = df_resampled_temp.fillna(method='ffill')
        tempSeries = calculate_average(df_resampled_temp_filled)
        listSeries.append(tempSeries)


    dataframe_resultante = pd.concat(listSeries, axis=1)

    dataframe_resultante.columns = colunas

    return dataframe_resultante





In [8]:
metaR_silver = defineIndex (metaR_silver)

In [9]:
mediaMetaR_silver = pipemediaTempoMetar(metaR_silver,['dew_point', 'windDirection', 'windVelocity', 'visibility','pressure', 'weather', 'sky', 'skyFeet'])
metaR_silver = metaR_silver.drop(['dew_point', 'windDirection', 'windVelocity', 'visibility','pressure', 'weather', 'sky', 'skyFeet'],axis=1)

C:\Users\Gaspar\AppData\Local\Temp\ipykernel_14420\666760620.py:22: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_resampled_temp_filled = df_resampled_temp.fillna(method='ffill')


In [10]:
novo_dataframe = pd.merge(metaR_silver, mediaMetaR_silver, left_index=True, right_index=True)


In [15]:
novo_dataframe = novo_dataframe.sort_values(by=['hora','aero'])

In [16]:
novo_dataframe

,hora,metaf,aero,station,type,metar_hora_formated,station,a few clouds,a few cumulonimbus,broken clouds,overcast,scattered clouds,dew_point,windDirection,windVelocity,visibility,pressure,weather,sky,skyFeet
hora_formated,,,,,,,,,,,,,,,,,,,,
2022-05-31 09:00:00,1654041600000,METAF SBGL 010000Z 25002KT 3000 BR OVC033...,SBGL,SBGL,routine report (automatic report),Thu Aug 31 21:00:00 2023,23.0,False,False,False,True,False,15.750000,151.875,7.750000,2500.000000,1016.500000,NaN,NaN,3300.000000
2022-05-31 09:00:00,1654041600000,METAF SBGR 010000Z 12008KT 1000 BR OVC033...,SBGR,SBGR,routine report (automatic report),Thu Aug 31 21:00:00 2023,15.0,False,False,False,True,False,15.750000,151.875,7.750000,2500.000000,1016.500000,NaN,NaN,3300.000000
2022-05-31 09:00:00,1654041600000,METAF SBKP 010000Z 12013KT 5000 BR BKN033...,SBKP,SBKP,routine report (automatic report),Thu Aug 31 21:00:00 2023,17.0,False,False,True,False,False,15.750000,151.875,7.750000,2500.000000,1016.500000,NaN,NaN,3300.000000
2022-05-31 09:00:00,1654041600000,METAF SBSP 010000Z 13008KT 1000 BR OVC033...,SBSP,SBSP,routine report (automatic report),Thu Aug 31 21:00:00 2023,15.0,False,False,False,True,False,15.750000,151.875,7.750000,2500.000000,1016.500000,NaN,NaN,3300.000000
2022-05-31 10:00:00,1654045200000,METAF SBGL 010100Z 02001KT 2000 BR OVC011 ...,SBGL,SBGL,"routine report, cycle 1 (automatic report)",Thu Aug 31 22:00:00 2023,22.0,False,False,False,True,False,16.333333,82.500,5.333333,1333.333333,1018.000000,NaN,NaN,2566.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-01 05:00:00,1677700800000,METAF SBGR 012000Z 12004KT 9999 RA FEW033...,SBGR,SBGR,"routine report, cycle 20 (automatic report)",Fri Sep 01 17:00:00 2023,27.0,True,False,False,False,False,18.333333,135.000,4.166667,1412.500000,1015.833333,NaN,NaN,2983.333333
2023-03-01 05:00:00,1677700800000,METAF SBSP 012000Z 13007KT 9999 -TSRA SCT02...,SBSP,SBSP,"routine report, cycle 20 (automatic report)",Fri Sep 01 17:00:00 2023,26.0,False,False,False,False,True,18.333333,135.000,4.166667,1412.500000,1015.833333,NaN,NaN,2983.333333
2023-03-01 06:00:00,1677704400000,METAF SBGR 012100Z 11006KT 9999 -TSRA SCT02...,SBGR,SBGR,"routine report, cycle 21 (automatic report)",Fri Sep 01 18:00:00 2023,26.0,False,False,False,False,True,18.500000,127.500,3.833333,1912.500000,1016.166667,NaN,NaN,2866.666667
